In [1]:
!pip install ultralytics opencv-python streamlit onnx onnxruntime onnxslim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import zipfile
import os
import shutil
from PIL import Image
import numpy as np
import torch
from ultralytics import YOLO
import tempfile

# Load Pytorch models
Hyena_model = YOLO('/content/hyena.pt')
Fishcat_model = YOLO('/content/fishcat.pt')
Classifier_model = YOLO('/content/classifier.onnx', task='detect')

def preprocess_image(image_path, input_size=640):
    """
    Preprocess the image to match YOLO and Classifier model requirements.
    Resize to 640x640 as required by the classifier model.
    """
    image = Image.open(image_path).convert("RGB").resize((input_size, input_size))
    image_array = np.array(image, dtype=np.float32) / 255.0  # Normalize pixel values
    image_array = np.transpose(image_array, (2, 0, 1))  # [H, W, C] -> [C, H, W]
    image_tensor = torch.tensor(image_array).unsqueeze(0)  # Add batch dimension
    return image_tensor

def classify_with_onnx(file_path, model):
    class_names = ["Striped_Hyena", "Fishing_Cat"]
    results = model(file_path)

    any_detections = False
    contains_fishing_cat = False
    contains_striped_hyena = False

    for result in results:
        detections_by_class = {name: [] for name in class_names}
        for i, box in enumerate(result.boxes):
            cls = int(box.cls.item())
            conf = box.conf.item()
            xyxy = box.xyxy.cpu().numpy().tolist()[0]

            if conf > 0.5:
                class_name = class_names[cls] if cls < len(class_names) else "Unknown"
                detections_by_class[class_name].append(xyxy)
                any_detections = True

                if class_name == "Fishing_Cat": contains_fishing_cat = True
                elif class_name == "Striped_Hyena": contains_striped_hyena = True

    if contains_striped_hyena: return "hyena"
    elif contains_fishing_cat: return "fishcat"
    else: return "undetected"

def classify_with_yolo(image_path, model, input_size = 224):
    input_tensor = preprocess_image(image_path, input_size)

    # Perform inference
    with torch.no_grad():
      results = model(input_tensor)  # Ultralytics model returns a Results object
    # Extract the predicted class index and name

    predicted_class_idx = results[0].probs.data.argmax().item()  # Get the class index
    predicted_class_name = model.names[predicted_class_idx]  # Map index to class name

    return predicted_class_name

def process_zip_and_classify(uploaded_zip):
    """
    Process the ZIP file, classify images, and return the resulting ZIP file with organized folders.
    """
    # Temporary directory to store unzipped contents
    with tempfile.TemporaryDirectory() as temp_dir:
        # Unzip the uploaded file into the temp directory
        with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Check if the unzipped folder contains a top-level folder (which matches the ZIP file name)
        # Get the list of files and folders in the temporary directory
        extracted_items = os.listdir(temp_dir)

        # If there's a single folder, move its contents directly into temp_dir
        if len(extracted_items) == 1 and os.path.isdir(os.path.join(temp_dir, extracted_items[0])):
            extracted_folder = os.path.join(temp_dir, extracted_items[0])
            # Move all contents of the extracted folder to temp_dir
            for item in os.listdir(extracted_folder):
                shutil.move(os.path.join(extracted_folder, item), temp_dir)
            # Remove the now-empty folder
            shutil.rmtree(extracted_folder)

        # Create folders for hyena, fishcat, and undetected
        os.makedirs(os.path.join(temp_dir, 'hyena'), exist_ok=True)
        os.makedirs(os.path.join(temp_dir, 'fishcat'), exist_ok=True)
        os.makedirs(os.path.join(temp_dir, 'undetected'), exist_ok=True)

        # Process each image in the folder
        for root, dirs, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root, file)
                if file.lower().endswith(('png', 'jpg', 'jpeg')):
                    # First, classify using the Classifier_model (ONNX)
                    group = classify_with_onnx(file_path, Classifier_model)

                    # Move image to respective folder
                    if group == "hyena":
                        target_folder = os.path.join(temp_dir, 'hyena')
                        subfolder = classify_with_yolo(file_path, Hyena_model)
                    elif group == "fishcat":
                        target_folder = os.path.join(temp_dir, 'fishcat')
                        subfolder = classify_with_yolo(file_path, Fishcat_model)
                    else:
                        target_folder = os.path.join(temp_dir, 'undetected')
                        subfolder = None

                    # Create sub-folder based on YOLO result (if any)
                    if subfolder:
                        subfolder_path = os.path.join(target_folder, subfolder)
                        os.makedirs(subfolder_path, exist_ok=True)
                        shutil.move(file_path, os.path.join(subfolder_path, file))
                    else:
                        shutil.move(file_path, os.path.join(target_folder, file))

        # Create a ZIP file containing the organized folders
        zip_filename = "/content/classified_images.zip"
        shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', temp_dir)

        return zip_filename

# Streamlit UI
st.title("Hyena or Fishcat Classifier")

uploaded_file = st.file_uploader("Upload a ZIP file or folder containing images", type=["zip"])

if uploaded_file and st.button("Process"):
    if uploaded_file:
        # Process the uploaded ZIP file and classify images
        zip_filename = process_zip_and_classify(uploaded_file)

        # Provide download link for the resulting ZIP file
        with open(zip_filename, "rb") as f:
            st.download_button("Download Classified ZIP", f, file_name="classified_images.zip", mime="application/zip")
    else:
        st.warning("Please upload a ZIP file.")


Writing app.py


In [3]:
!curl https://loca.lt/mytunnelpassword

34.69.14.203

In [5]:
!streamlit run app.py &>/content/logs.txt &
import subprocess
import re

# Start localtunnel and capture its output
localtunnel_process = subprocess.Popen(
    ["npx", "localtunnel", "--port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

for line in localtunnel_process.stdout:
    print(line, end="")


your url is: https://cuddly-pots-enjoy.loca.lt


KeyboardInterrupt: 